In [1]:
] activate .

 Activating environment at `~/projects/mjolnir/Mjolnir/examples/futamura/Project.toml`


# Futamura Projections in Julia

We're going to implement the first [Futamura Projection](https://en.wikipedia.org/wiki/Partial_evaluation#Futamura_projections) in Julia. The goal is to compile code without writing a compiler – we'll only write an interpreter, which is much easier to get working, and then *specialise* it to create a compiled program.

The language we're interpreting is [Brainfuck](https://en.wikipedia.org/wiki/Brainfuck), which is helpful because it's particularly simple to implement. The interpreter itself isn't too important (though it's fun to write one yourself), so I'm going to [pull in one I made earlier](https://github.com/MikeInnes/Mjolnir.jl/blob/78f5b5614dbab7a90463ccf409b50ad2816e9662/examples/futamura/brainfuck.jl).

In [2]:
include("brainfuck.jl")

We separate parsing and execution, so we can parse BF into an AST like this:

In [3]:
bfparse("+[><]-")

3-element Array{Union{BFInstruction, Loop},1}:
 inc::BFInstruction = 0
 Loop(Union{BFInstruction, Loop}[right, left])
 dec::BFInstruction = 1

And run it like this:

In [4]:
interpret("[->>+<<]>[->+<]", [5, 8, 0])

3-element Array{Int64,1}:
  0
  0
 13

The program `[->>+<<]>[->+<]` implements addition of the first two cells in the tape ($5$ and $8$), putting the results in cell 3 ($13$). Here's a similar implementation of multiplication, using a nested loop.

In [5]:
interpret("[->[->>+<<]>>[-<+<+>>]<<<]", [5, 8, 0, 0])

4-element Array{Int64,1}:
  0
  8
 40
  0

Now we'll bring in Mjolnir, a partial evaluation system, to specialise the `interpret` function on the input program. This is very simple. We provide the input string as a constant, and the tape as a vector with unknown values (we could pass an array of zeros, but this would cause the whole program to be evaluated at compile time, which would be less interesting).

In [6]:
using Mjolnir

@trace interpret("++", Vector{Int64})

1: (%1 :: const(interpret), %2 :: const(++), %3 :: Array{Int64,1})
  %4 = (getindex)(%3, 1) :: Int64
  %5 = (+)(%4, 1) :: Int64
  %6 = (setindex!)(%3, %5, 1) :: Array{Int64,1}
  %7 = (getindex)(%3, 1) :: Int64
  %8 = (+)(%7, 1) :: Int64
  %9 = (setindex!)(%3, %8, 1) :: Array{Int64,1}
  return %3

Mjolnir works with IR, not high-level Julia code, so this is a bit hard to read. But you can hopefully see that there's no parsing happening (there's no need, since it can all be done at compile time). We can show that it's equivalent to the following hand-written code representing the brainfuck program – the IR is the same (down to some formatting differences).

In [7]:
function test(_, tape)
    tape[1] += 1
    tape[1] += 1
    return tape
end

@code_typed optimize=false test(nothing, Int[])

CodeInfo(
1 ─ %1 = Base.getindex(tape, 1)::Int64
│   %2 = (%1 + 1)::Int64
│        Base.setindex!(tape, %2, 1)::Any
│   %4 = Base.getindex(tape, 1)::Int64
│   %5 = (%4 + 1)::Int64
│        Base.setindex!(tape, %5, 1)::Any
└──      return tape
) => Array{Int64,1}

How did this happen? At core our interpreter is [just a loop](https://github.com/MikeInnes/Mjolnir.jl/blob/78f5b5614dbab7a90463ccf409b50ad2816e9662/examples/futamura/brainfuck.jl#L57-L71) which checks each brainfuck instruction and runs the corresponding Julia code. But we know how many steps the interpreter loop needs to run for (twice, one for each `+` in our program) so we can unroll it. We also know what each instruction is (`+`), so we can get rid of the `if`/`else` and just insert the right instruction directly.

This works for loops too:

In [8]:
@trace interpret("[->>+<<]>[->+<]", Vector{Int})

1: (%1 :: const(interpret), %2 :: const([->>+<<]>[->+<]), %3 :: Array{Int64,1})
  %4 = (getindex)(%3, 1) :: Int64
  %5 = (!=)(%4, 0) :: Bool
  br 3 unless %5
  br 2
2:
  %6 = (getindex)(%3, 1) :: Int64
  %7 = (-)(%6, 1) :: Int64
  %8 = (setindex!)(%3, %7, 1) :: Array{Int64,1}
  %9 = (getindex)(%3, 3) :: Int64
  %10 = (+)(%9, 1) :: Int64
  %11 = (setindex!)(%3, %10, 3) :: Array{Int64,1}
  %12 = (getindex)(%3, 1) :: Int64
  %13 = (!=)(%12, 0) :: Bool
  br 3 unless %13
  br 2
3:
  %14 = (getindex)(%3, 2) :: Int64
  %15 = (!=)(%14, 0) :: Bool
  br 5 unless %15
  br 4
4:
  %16 = (getindex)(%3, 2) :: Int64
  %17 = (-)(%16, 1) :: Int64
  %18 = (setindex!)(%3, %17, 2) :: Array{Int64,1}
  %19 = (getindex)(%3, 3) :: Int64
  %20 = (+)(%19, 1) :: Int64
  %21 = (setindex!)(%3, %20, 3) :: Array{Int64,1}
  %22 = (getindex)(%3, 2) :: Int64
  %23 = (!=)(%22, 0) :: Bool
  br 5 unless %23
  br 4
5:
  return %3

Again, this is a bit verbose and hard to read, but it's equivalent to the Julia program

```julia
while tape[1] != 0
    tape[1] -= 1
    tape[3] += 1
end
while tape[2] != 0
    tape[2] -= 1
    tape[3] += 1
end
```

which is a direct translation of the original program `[->>+<<]>[->+<]`. (There's also `ptr` variable in the interpreter, tracking the current location of the brainfuck pointer, but in this case we can elide that too.) Both `while` loops here are 'really' the same `while` loop [inside our interpreter](https://github.com/MikeInnes/Mjolnir.jl/blob/78f5b5614dbab7a90463ccf409b50ad2816e9662/examples/futamura/brainfuck.jl#L50-L52), but the loop gets specialised twice for both the loops in our brainfuck code. Similarly, our multiplication program `[->[->>+<<]>>[-<+<+>>]<<<]` gets compiled to the lowered version of

```julia
while tape[1] != 0
    tape[1] -= 1
    while tape[2] != 0
        tape[2] -= 1
        tape[4] += 1
    end
    while tape[4] != 0
        tape[4] -= 1
        tape[2] += 1
        tape[3] += 1
    end
end
```

Aside from just admiring the skinny code that gets generated, we can of course compile the output via Julia and run it to make sure it's doing the right thing.

In [9]:
using IRTools

ir = @trace interpret("[->[->>+<<]>>[-<+<+>>]<<<]", Vector{Int})

f = IRTools.func(ir)

f(nothing, nothing, [5, 8, 0, 0])

4-element Array{Int64,1}:
  0
  8
 40
  0

Of course, being compiled code, it's significantly faster than the uncompiled version – by ~40x on my system.

In [10]:
using BenchmarkTools

@btime interpret("[->[->>+<<]>>[-<+<+>>]<<<]", [5, 8, 0, 0]);
@btime $f(nothing, nothing, [5, 8, 0, 0]);

  3.286 μs (59 allocations: 2.06 KiB)
  87.863 ns (1 allocation: 112 bytes)


Let's make a wrapper that compiles brainfuck code to a Julia function (by specialising our interpreter and then compiling the result through Julia).

In [11]:
using Mjolnir: Defaults, Const, trace

function compile(bf)
    f = trace(Defaults(),Const(interpret),Const(bf),Vector{Int}) |> IRTools.func
    (a, b) -> f(nothing, nothing, NVector((a, b, 0, 0)))[3]
end

compile (generic function with 1 method)

This utility uses `NVector`, which is easy for Julia and LLVM to optimise – that makes this code about 20x faster still.

In [12]:
f = compile("[->[->>+<<]>>[-<+<+>>]<<<]")

@btime $f(5, 8);

  4.505 ns (0 allocations: 0 bytes)


This gets even better for our addition code, which is about four million times faster on this benchmark.

In [13]:
f = compile("[->>+<<]>[->+<]")

@btime interpret("[->>+<<]>[->+<]", NVector((2^20, 2^20, 0, 0)));
@btime $f(2^20, 2^20);

  31.418 ms (34 allocations: 1.11 KiB)
  8.194 ns (0 allocations: 0 bytes)


Single-digit nanoseconds might seem implausibly fast, but it's really just the result of LLVM recognising that our compiled loop is equivalent to an addition. We can check out the optimised LLVM output to see the single, native machine add instruction that this code boils down to.

In [14]:
@code_llvm f(1, 1)


;  @ In[11]:5 within `#5'
define i64 @"julia_#5_6821"(i64, i64) {
top:
; ┌ @ /Users/mike/projects/mjolnir/Mjolnir/examples/futamura/brainfuck.jl:83 within `NVector' @ /Users/mike/projects/mjolnir/Mjolnir/examples/futamura/brainfuck.jl:83
; │┌ @ essentials.jl:309 within `convert'
    %2 = add i64 %0, %1
; └└
  ret i64 %2
}


Remember, this is not the result of compiling a brainfuck program, but the result of optimising an interpreter that parses and then executes brainfuck programs. By partially evaluating that interpeter, and feeding the result through Julia/LLVM, we get a relatively capable, optimising brainfuck compiler without having to actually write one.